In [1]:
%matplotlib inline

http://alleninstitute.github.io/AllenSDK/_static/examples/nb/mouse_connectivity.html

In [2]:
import pandas as pd
import numpy as np
import nibabel as nb
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from nipype.interfaces import fsl

### Download second level substructures of isocortex

In [3]:
mcc = MouseConnectivityCache(manifest_file='/home/julia/data/gradients/allen_atlas/allen_manifest.json')

In [4]:
structure_tree = mcc.get_structure_tree()

In [5]:
iso_id = structure_tree.get_structures_by_name(['Isocortex'])[0]['id']
regions = structure_tree.children([iso_id])[0]

In [6]:
hdr = nb.load('/home/julia/data/gradients/allen_atlas/cortex_mask_tight.nii.gz').header
aff = nb.load('/home/julia/data/gradients/allen_atlas/cortex_mask_tight.nii.gz').affine
shape = nb.load('/home/julia/data/gradients/allen_atlas/cortex_mask_tight.nii.gz').shape

In [7]:
for region in regions:
    children = structure_tree.children([region['id']])[0]
    for child in children:
        if "layer" in child['name'].lower():
            pass
        elif child['id'] == 738: #structure is missing in API
            pass
        else:
            mask, _ = mcc.get_structure_mask(child['id'],'/tmp/%s.nrrd' % child['acronym'])
            nb.Nifti1Image(mask, aff, hdr).to_filename('/home/julia/data/gradients/allen_atlas/regions/%s.nii.gz'
                                               % child['acronym'])

In [8]:
# Start a list of subregions
finest = []
for region in regions:
    children = structure_tree.children([region['id']])[0]
    for child in children:
        if "layer" in child['name'].lower():
            pass
        elif child['id'] == 738:
            pass
        else:
            finest.append(child['acronym'])

In [9]:
# Add the level 1 regions that have no subregions
for region in regions:
    if True in [(region['acronym'] in f) for f in finest]:
        pass
    else:
        finest.append(region['acronym'])
        mask, _ = mcc.get_structure_mask(region['id'],'/tmp/%s.nrrd' % region['acronym'])
        nb.Nifti1Image(mask, aff, hdr).to_filename('/home/julia/data/gradients/allen_atlas/regions/%s.nii.gz'
                                               % region['acronym'])

In [10]:
# Add subregions of SSp
for ss in structure_tree.children([322])[0]:
    if "layer" in ss['name'].lower():
        pass
    else:
        finest.append(ss['acronym'])
        mask, _ = mcc.get_structure_mask(ss['id'], '/tmp/%s.nrrd' % ss['acronym'])
        nb.Nifti1Image(mask, aff, hdr).to_filename('/home/julia/data/gradients/allen_atlas/regions/%s.nii.gz'
                                            % ss['acronym'])
finest.remove('SSp')

In [11]:
df = pd.DataFrame(data=zip(range(1,len(finest)+1),finest),
                 columns=['id', 'acronym'])
df.to_csv('/home/julia/workspace/mouse_gradients/regions.csv')

### Make annotation volume

In [12]:
df = pd.read_csv('/home/julia/workspace/mouse_gradients/regions.csv')
df = df.drop(columns=["Unnamed: 0"])

In [13]:
# Make a single annotation volume
annot_finest = np.zeros(shape)

for a in range(len(df['acronym'])):
    mask = nb.load("/home/julia/data/gradients/allen_atlas/regions/%s.nii.gz" % df['acronym'][a]).get_data()
    annot_finest[mask == 1] = df['id'][a]

In [14]:
nb.save(nb.Nifti1Image(np.round(annot_finest), aff, hdr), "/home/julia/data/gradients/allen_atlas/annot_finest.nii.gz")

### Downsample annotation volume

In [15]:
resamp = fsl.FLIRT(in_file="/home/julia/data/gradients/allen_atlas/annot_finest.nii.gz",
                   reference="/home/julia/data/gradients/allen_atlas/annot_finest.nii.gz",
                   apply_isoxfm=0.2,
                   interp="nearestneighbour",
                   out_file="/home/julia/data/gradients/allen_atlas/annot_finest_200um.nii.gz")
resamp.run()

In [16]:
rounded = np.round(nb.load("/home/julia/data/gradients/allen_atlas/annot_finest_200um.nii.gz").get_data())
hdr_200 = nb.load("/home/julia/data/gradients/allen_atlas/annot_finest_200um.nii.gz").header
aff_200 = nb.load("/home/julia/data/gradients/allen_atlas/annot_finest_200um.nii.gz").affine
nb.save(nb.Nifti1Image(rounded, aff_200, hdr_200), 
        "/home/julia/data/gradients/allen_atlas/annot_finest_200um.nii.gz")